IEEE Access. Geometric deep lean learning

## Packages

### Import necessary packages

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import re
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from collections import Counter

### Set Package parameters

In [ ]:
plt.style.use('seaborn')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Calibri'
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.labelspacing'] = 0.3
sns.set_palette('muted')

np.set_printoptions(edgeitems=30, 
                    linewidth=100000, 
                    formatter=dict(float=lambda x: "%.3g" % x))

## Data pre--processing

### Load collected re--tweets

In [ ]:
retweets_df = pd.read_pickle('all_retweets.pkl')

### Select the relevant information

In [ ]:
filtered_tweets = retweets_df[['retweeter_user_id', 
                               'retweeted_user_id', 
                               'hashtag', 
                               'group', 
                               'created_at', 
                               'text']]

### Clean and balance data--set

In [ ]:
"""Delete the first and last 6 hours of tweets 
as this is the time it took to collect all tweets"""

filtered_tweets = filtered_tweets[(filtered_tweets['created_at'] > min(filtered_tweets['created_at']) + datetime.timedelta(hours=6)) & (filtered_tweets['created_at'] < max(filtered_tweets['created_at']) - datetime.timedelta(hours=6))]

### Select hastag group #machinelearning

In [ ]:
filtered_tweets = filtered_tweets[filtered_tweets['hashtag'] == '#machinelearning']

## Hyper--parameter design

### Temporal depth

In [ ]:
min_time = min(filtered_tweets['created_at'])
max_time = max(filtered_tweets['created_at']) 
time_length = max_time - min_time
print('Start time',min_time,'End time',max_time,'duration',time_length)

time_depth = 6

t = time_depth/7

time_end_training = min_time + time_length * t
print(time_end_training)
print('Duration training',time_end_training-min_time)

time_start_test = time_end_training
time_end_test = min_time + time_length
print(time_end_test)
print('Duration test',time_end_test-time_start_test)

### Spatial depth

In [ ]:
"""Three layers with weights 
that are initialized with weights between 0 and 0.1"""

In [ ]:
np.random.seed(42)
weights_1 = np.random.rand(len(training_ids),len(training_ids))*0.1
weights_2 = np.random.rand(len(training_ids),len(training_ids))*0.1
weights_3 = np.random.rand(len(training_ids),len(training_ids))*0.1

## Filter Data--set

In [ ]:
training_tweets = filtered_tweets[filtered_tweets['created_at'] <= time_end_training].reset_index()

test_tweets = filtered_tweets[(filtered_tweets['created_at'] > time_end_training) & (filtered_tweets['created_at'] <= time_end_test)].reset_index()

training_ids = set(training_tweets['retweeted_user_id'].tolist()+training_tweets['retweeter_user_id'].tolist())
print(len(training_ids),'unique IDs for the training retweets')

test_ids = set(test_tweets['retweeted_user_id'].tolist()+test_tweets['retweeter_user_id'].tolist())
print(len(test_ids),'unique IDs for all of the test retweets')

used_ids = list(training_ids)

used_ids.sort(key=int)

filtered_test_tweets = test_tweets[test_tweets['retweeted_user_id'].isin(training_ids) & test_tweets['retweeter_user_id'].isin(training_ids)].copy()

## Re--tweet content cleaning

In [ ]:
def string_alphabet_to_np_float(string):
    '''String that contains only alphabetics to float between >0 and 1'''
    cleaned_string = clean_str(string)
    return (np.frombuffer(cleaned_string.encode(), np.int8) - ord('a') + 1)/26

In [ ]:
def clean_str(string):
    '''Only letters from string'''
    string = re.sub(r"\\", "", string)    
    string = re.sub(r"\'", "", string)    
    string = re.sub(r"\"", "", string)
    string = re.sub(r'[\W_]+', '', string)
    string = re.sub(r'[0-9]', '', string)
    return string.strip().lower()

In [ ]:
filtered_validation_tweets = validation_tweets[validation_tweets['retweeted_user_id'].isin(training_ids) & validation_tweets['retweeter_user_id'].isin(training_ids)].copy()

In [ ]:
"""To make the access to the weights easier, 
the IDs are changed to an index starting at 0"""

training_tweets['retweeter_user_index'] = training_tweets['retweeter_user_id'].apply(used_ids.index)

training_tweets['retweeted_user_index'] = training_tweets['retweeted_user_id'].apply(used_ids.index)

filtered_test_tweets['retweeter_user_index'] = filtered_test_tweets['retweeter_user_id'].apply(used_ids.index)

filtered_test_tweets['retweeted_user_index'] = filtered_test_tweets['retweeted_user_id'].apply(used_ids.index)

## Re--tweet content transformation

### Naive Bayes sentiment analysis

In [ ]:
blob_object = TextBlob(filtered_tweets['text'][502450], 
                       analyzer=NaiveBayesAnalyzer())

print('How positiv is the sentiment of the previous tweet?',
      blob_object.sentiment.p_pos)

In [ ]:
def get_positive_sentiment_from_text(text, length):
    '''Get positive sentiment from text. Minimum is set to 0.001. Maximum is 1'''
    blob_object = TextBlob(text, analyzer=NaiveBayesAnalyzer())
    return max(blob_object.sentiment.p_pos,0.001)

In [ ]:
def get_positive_sentiment_and_categories_from_text(text, length):
    '''Get positive sentiment from text. Minimum is set to 0.001. Maximum is 1'''
    blob_object = TextBlob(text, analyzer=NaiveBayesAnalyzer())
            
    floats_of = string_alphabet_to_np_float(text)
    if floats_of.shape[0] < 240:
        floats_of = np.concatenate((floats_of, np.array([0] * (240 - len(floats_of)))), axis=0)

    categories = model.predict(np.array( [floats_of,] ))
    ad = [max(blob_object.sentiment.p_pos,0.001)] + list(categories[0])
    return np.array(ad)

### Most used words vector

In [ ]:
most_used_words = Counter()
for text in training_tweets['text']:
    for t in text.lower().split():
        most_used_words[t] += 1
        
top_used = most_used_words.most_common(10)
top_used_names = [t[0] for t in top_used]
top_used_values = [t[1] for t in top_used]

## Reduced Lapalacian

In [ ]:
def create_adjacency_dict(df, direction):
    '''Create adjacency dictionary through the connections created by retweets'''
    connection_one = defaultdict(set)
    for i in range(len(df)):
        if direction == 'undirected' or direction == 'retweeted':
            connection_one[df.iloc[i]['retweeter_user_index']].add(df.iloc[i]['retweeted_user_index'])
        if direction == 'undirected' or direction == 'retweeter':
            connection_one[df.iloc[i]['retweeted_user_index']].add(df.iloc[i]['retweeter_user_index'])
    return connection_one

In [ ]:
def get_nth_neighbour_for_adjacency_dict(direct_neighbours, neighbour_depth):
    '''Calculate the nth neighbour of a network'''
    new_one = deepcopy(direct_neighbours)
    new_dict = deepcopy(direct_neighbours)
    
    for n in range(neighbour_depth - 1):
        for node, connected in new_one.items():
            for con in connected:
                if con in direct_neighbours:
                    new_dict[node].update(direct_neighbours[con])
            new_dict[node].discard(node)
        new_one = deepcopy(new_dict)
    return new_dict

In [ ]:
def adjacency_dict_to_matrix(base_nodes, adj_dict, directed):
    '''Create adjacency matrix from adjacency dictionary'''
    if directed:
        new_graph = nx.DiGraph()
    else:
        new_graph = nx.Graph()
    for node in base_nodes:
        new_graph.add_node(node)
    for node, connected in adj_dict.items():
        for con in connected:
            new_graph.add_edge(node, con, weight=1)
    adjacency_matrix = nx.adjacency_matrix(new_graph)
    return adjacency_matrix

In [ ]:
def undirected_adjacency_dict_to_normalized_laplacian(base_nodes, adj_dict):
    '''Create normalized laplacian from adjacency dictionary'''
    new_graph = nx.Graph()
    
    for node in base_nodes:
        new_graph.add_node(node)

    for node, connected in adj_dict.items():
        for con in connected:
            new_graph.add_edge(node, con, weight=1)
    normalizedlaplacian_matrix = nx.normalized_laplacian_matrix(new_graph)
    return normalizedlaplacian_matrix

In [ ]:
def undirected_adjacency_dict_to_subclusters(adj_dict, number_of_clusters, only_if_both_here):
    '''Create normalized laplacian from adjacency dictionary'''
    new_graph = nx.Graph()
    
    node_to_group = dict()
    
    for node, connected in adj_dict.items():
        for con in connected:
            if len(only_if_both_here) > 0:
                if node in only_if_both_here and con in only_if_both_here:
                    new_graph.add_edge(node, con, weight=1)
            else:
                new_graph.add_edge(node, con, weight=1)
    
    sub_graphs = [new_graph.subgraph(c) for c in nx.connected_components(new_graph)]
    
    for sub_graph in sub_graphs[1:]:
        for node in sub_graph:
            node_to_group[node] = number_of_clusters

    adj_matrix = nx.to_numpy_matrix(sub_graphs[0]) #Converts graph to an adj matrix with adj_matrix[i][j] represents weight between node i,j.
    node_list = list(sub_graphs[0].nodes()) #returns a list of nodes with index mapping with the a 

    clusters = SpectralClustering(affinity = 'precomputed', assign_labels="kmeans",random_state=0,n_clusters=number_of_clusters).fit_predict(adj_matrix)
    
    for i, node in enumerate(node_list):
        node_to_group[node] = clusters[i]
    #print('Clusters',len(clusters))
    #for i in range(n):
    #    print("Subgraph:", i, "consists of ",len(sub_graphs[i].nodes()))
    return node_to_group

In [ ]:
def tweets_to_connection_matrix(tweets, ids_length):
    '''Create connection matrix from all tweets'''
    connections = np.zeros((ids_length, ids_length), dtype=np.bool)
    
    adj_dict = create_adjacency_dict(tweets, 'undirected')
    
    for a in adj_dict:
        for b in adj_dict[a]:
            connections[a,b] = 1
            connections[b,a] = 1
    
    return connections.flatten()

In [ ]:
"""Source Code
@article{dallamico_unified_2020,
	title = {A unified framework for spectral clustering in sparse graphs}
    url = {http://arxiv.org/abs/2003.09198}"""

def lap(A,classes, assortativity,number_eig):

    ''' This function performs spectral clustering on the best eigenvector of the random walk laplacian matrix D^{-1}A. 

    Use: y_kmeans,eigenvalues, X, ov = lap(A,classes, assortativity,number_eig)
    Inputs: A a (symmetric) n x n adjacency matrix of a graph, classes a vector of size n with the underlying ground truth class assignment and assortativity is set to 1 if one seeks assortative blocks or to -1 if one seeks disassortative blocks, number_eig is the number of eigenvalues among which look for the best 
    Outputs: y_kmeans the vector of size n of detected classes, eigenvalues the values of the two smallest (largest) eigenvalues of A, X the eigenvector of size n used for the classification and ov the overlap between the detected classes y_kmeans and the ground truth classes.

    '''

    n = len(A)
    A = A.astype(float)
    d = np.sum(A,axis = 0)
    D_1 = np.diag(d**(-1))
    L = np.dot(D_1,A)

    if assortativity > 0:
        eigenvalues,eigenvectors = scipy.sparse.linalg.eigs(L, number_eig, which='LR')
    else:
        eigenvalues,eigenvectors = scipy.sparse.linalg.eigs(L, number_eig, which='SR')

    eigenvalues = eigenvalues.real # the matrix is  symmetric so the eigenvalues are real
    eigenvectors = eigenvectors.real # the matrix is  symmetric so the eigenvectors are real
    idx = eigenvalues.argsort()[::-1] # order the eigenvectors
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]

    current_ov = 0
    y = np.zeros(n)
    vector = np.zeros((len(eigenvectors[:,0]),2))

    # This cycle picks the best out of number_eig computed eigenvectors

    for i in range(1,number_eig):

        X = np.ones(len(eigenvectors[:,0]))
        X = np.column_stack((X,eigenvectors[:,i]))
        kmeans = KMeans(n_clusters = 2)
        kmeans.fit(X)
        y_kmeans = kmeans.predict(X)
        precision1 = overlap(y_kmeans,classes) # choose between the class assignment 0 -> A, 1 -> B and 0 -> B, 1 -> A, A and B being the two classes. This keeps the overlap positive
        precision2 = overlap(1-y_kmeans,classes)
        ov = max(precision1, precision2)
        if ov > current_ov:
            current_ov = ov
            y = y_kmeans
            vector = np.zeros(len(eigenvectors[:,0]))
            vector = np.column_stack((vector,eigenvectors[:,i]))
            if ov == precision2:
                y = 1-y_kmeans

    return y,eigenvalues, vector, current_ov

## Re-tweet search

In [ ]:
def find_tweet(df, user_index_a, user_index_b):
    '''Find tweet between both users'''
    return df[((df['retweeter_user_index'] == user_index_a) & (df['retweeted_user_index'] == user_index_b)) | ((df['retweeted_user_index'] == user_index_a) & (df['retweeter_user_index'] == user_index_b))]['text'].values

In [ ]:
def find_tweet_only_from(df, user_index_b):
    '''Find tweet from that user'''
    return df[(df['retweeted_user_index'] == user_index_b) | (df['retweeter_user_index'] == user_index_b)]['text'].values

In [ ]:
def find_all_tweet_connected_to(df, user_indexes_b):
    '''Find tweet from that user'''
    return df[(df['retweeted_user_index'].isin(user_indexes_b)) | (df['retweeter_user_index'].isin(user_indexes_b))]['text'].values

## Re-tweet content length normalization

In [ ]:
def array_of_strings_to_float_with_length(strings, needed_length):
    '''Take all strings. Transform these into floats and adapt them, to fit a specified length'''
    new_strings = np.array([])
    for i in range(strings.shape[0]):
        conv_str = string_alphabet_to_np_float(strings[i])
        if new_strings.shape[0] == 0:
            new_strings = np.transpose(get_needed_length_of_array(conv_str, needed_length))
            new_strings = new_strings.reshape(new_strings.shape[0], 1)
        else:
            new_strings = np.concatenate((new_strings, np.transpose(get_needed_length_of_array(conv_str, needed_length)).reshape(new_strings.shape[0], 1)), axis=1)
    return new_strings

In [ ]:
"""Transform re--tweet conent to constant for research purposes"""

def return_constant_string(length):
    return np.array([0.5]*length)

In [ ]:
def array_of_strings_to_float_with_length_random(strings, needed_length):
    '''Take all strings. Transform these into floats and adapt them, to fit a specified length'''
    new_strings = np.array([])
    for i in range(strings.shape[0]):
        conv_str = return_random_string(240)
        if new_strings.shape[0] == 0:
            new_strings = np.transpose(get_needed_length_of_array(conv_str, needed_length))
            new_strings = new_strings.reshape(new_strings.shape[0], 1)
        else:
            new_strings = np.concatenate((new_strings, np.transpose(get_needed_length_of_array(conv_str, needed_length)).reshape(new_strings.shape[0], 1)), axis=1)
    return new_strings

In [ ]:
def array_of_strings_to_float_with_length_sentiment_and_categories(strings, needed_length):
    '''Take all strings. Transform these into sentiment and categories and adapt them, to fit a specified length'''
    new_strings = np.array([])
    for i in range(strings.shape[0]):
        conv_str = get_positive_sentiment_and_categories_from_text(strings[i], 240)
        if new_strings.shape[0] == 0:
            new_strings = np.transpose(get_needed_length_of_array(conv_str, needed_length))
            new_strings = new_strings.reshape(new_strings.shape[0], 1)
        else:
            new_strings = np.concatenate((new_strings, np.transpose(get_needed_length_of_array(conv_str, needed_length)).reshape(new_strings.shape[0], 1)), axis=1)
    return new_strings

In [ ]:
def get_needed_length_of_array(arr, needed_length):
    '''Adapt array to specific length. If it is too short, duplicate it and if it is too long, cut it'''
    if arr.shape[0] >= needed_length:
        return arr[:needed_length]
    else:
        repeats_needed = (needed_length // arr.shape[0]) + 1
        tiled = np.tile(arr, repeats_needed)
        return tiled[:needed_length]

## Geometric deep lean learning

### Activation function for convolution

In [ ]:
def relu(array):
    return np.maximum(0, array)

def relu_backwards(array, Z):
    dZ = np.array(array, copy = True)
    dZ[Z <= 0] = 0
    return dZ

### Performance measurement

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return(TP, FP, TN, FN)

### Node cluster

In [ ]:
cluster_to_nodes = [[] for i in range(11)]
for key in node_to_cluster.keys():
    cluster_to_nodes[node_to_cluster[key]].append(key)

### Geometric deep lean learning convolution

In [ ]:
"""
The neural network is based on multiple layers. 
Layer one considers all neighbours with distance 1 from node a. 
Layer two all neighbours with distance 2 from node a and layer 
three all neighbours with distance 3. 
The training is done by minimizing the loss between the results 
of the third and second layer and then of the second with 
the first layer. These steps are performed multiple time for all results.
"""

In [ ]:
%%time
time_length = 5
learning_rate = 0.08
global_iterations = 5
local_iterations = 50
validation_losses = []
weights_changes = []
sum_of_all_losses = 0

weights_changes.append(weights_1.copy())

maximum_minutes_validation = int((max(filtered_validation_tweets['created_at']) - min(filtered_validation_tweets['created_at'])).total_seconds()/60)
for cluster in cluster_to_nodes:
    for minutes_started in range(0, maximum_minutes_validation, time_length):
        time_slice = filtered_validation_tweets[(filtered_validation_tweets['created_at'] < min(filtered_validation_tweets['created_at']) + datetime.timedelta(minutes=minutes_started+time_length)) & (filtered_validation_tweets['created_at'] >= min(filtered_validation_tweets['created_at']) + datetime.timedelta(minutes=minutes_started))]

        time_slice = time_slice[time_slice['retweeted_user_index'].isin(cluster) & time_slice['retweeter_user_index'].isin(cluster)]
        
        if len(time_slice) == 0:
            continue
            
        all_ids = list(set(time_slice['retweeted_user_index'].tolist() + time_slice['retweeter_user_index'].tolist()))

        adj_dict_n1 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 1)
        adj_matrix_n1 = adjacency_dict_to_matrix(all_ids, adj_dict_n1, True)

        adj_dict_n2 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 2)
        adj_matrix_n2 = adjacency_dict_to_matrix(all_ids, adj_dict_n2, True)

        laplacian_n1 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n1)
        eigen_values_n1, eigen_vectors_n1 = np.linalg.eigh(laplacian_n1.toarray())

        laplacian_n2 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n2)
        eigen_values_n2, eigen_vectors_n2 = np.linalg.eigh(laplacian_n2.toarray())

        for a in all_ids:
            if len(adj_dict_n1[a]) == 0:
                continue

            za_parts = []
            needed_tweets_n1 = find_all_tweet_connected_to(time_slice, adj_dict_n1[a])
            conv_strings_n1 = array_of_strings_to_float_with_length_sentiment_and_categories(needed_tweets_n1, laplacian_n1.shape[0])

            na_za_n1 = np.array([weights_1[a, b]*np.matmul(array_n(laplacian_n1.toarray(),i), conv_strings_n1) for i, b in enumerate(adj_dict_n1[a])])
            na_za_n1 = sum(na_za_n1)
            za_n1 = relu(na_za_n1)

            na_za_n2 = [weights_2[a, b]*np.matmul(array_n(eigen_vectors_n2,i), za_n1) for i, b in enumerate(adj_dict_n2[a])]
            na_za_n2 = sum(na_za_n2)
            za_n2 = relu(na_za_n2)

            try:
                delta_loss = (za_n1 - za_n2) * relu_backwards(za_n2, na_za_n1)
                sum_of_all_losses += np.sum(np.abs(delta_loss))/(delta_loss.size)
            except:
                continue

print('loss',sum_of_all_losses)
validation_losses.append(sum_of_all_losses)

for global_iteration in range(global_iterations):
    sum_of_all_losses = 0

    for cluster in cluster_to_nodes:
        maximum_minutes_training = int((max(training_tweets['created_at']) - min(training_tweets['created_at'])).total_seconds()/60)
        for minutes_started in range(0, maximum_minutes_training, time_length):
            time_slice = training_tweets[(training_tweets['created_at'] < min(training_tweets['created_at']) + datetime.timedelta(minutes=minutes_started+time_length)) & (training_tweets['created_at'] >= min(training_tweets['created_at']) + datetime.timedelta(minutes=minutes_started))]

            time_slice = time_slice[time_slice['retweeted_user_index'].isin(cluster) & time_slice['retweeter_user_index'].isin(cluster)]
        
            if len(time_slice) == 0:
                continue

            all_ids = list(set(time_slice['retweeted_user_index'].tolist() + time_slice['retweeter_user_index'].tolist()))

            adj_dict_n1 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 1)
            adj_matrix_n1 = adjacency_dict_to_matrix(all_ids, adj_dict_n1, True)

            adj_dict_n2 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 2)
            adj_matrix_n2 = adjacency_dict_to_matrix(all_ids, adj_dict_n2, True)

            adj_dict_n3 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 3)
            adj_matrix_n3 = adjacency_dict_to_matrix(all_ids, adj_dict_n3, True)

            laplacian_n1 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n1)
            eigen_values_n1, eigen_vectors_n1 = np.linalg.eigh(laplacian_n1.toarray())

            laplacian_n2 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n2)
            eigen_values_n2, eigen_vectors_n2 = np.linalg.eigh(laplacian_n2.toarray())

            laplacian_n3 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n3)
            eigen_values_n3, eigen_vectors_n3 = np.linalg.eigh(laplacian_n3.toarray())

            for a in all_ids:
                if len(adj_dict_n1[a]) == 0:
                    continue

                za_parts = []
                needed_tweets_n1 = find_all_tweet_connected_to(time_slice, adj_dict_n1[a])
                conv_strings_n1 = array_of_strings_to_float_with_length_sentiment_and_categories(needed_tweets_n1, laplacian_n1.shape[0])

                na_za_n1 = np.array([weights_1[a, b]*np.matmul(array_n(laplacian_n1.toarray(),i), conv_strings_n1) for i, b in enumerate(adj_dict_n1[a])])
                na_za_n1 = sum(na_za_n1)
                za_n1 = relu(na_za_n1)

                na_za_n2 = [weights_2[a, b]*np.matmul(array_n(eigen_vectors_n2,i), za_n1) for i, b in enumerate(adj_dict_n2[a])]
                na_za_n2 = sum(na_za_n2)
                za_n2 = relu(na_za_n2)

                na_za_n3 = [weights_3[a, b]*np.matmul(array_n(eigen_vectors_n3,i), za_n2) for i, b in enumerate(adj_dict_n3[a])]
                na_za_n3 = sum(na_za_n3)
                za_n3 = relu(na_za_n3)

                for iterations in range(local_iterations):
                    delta_loss = (za_n3 - za_n2) * relu_backwards(za_n3, na_za_n2)

                    if np.sum(np.abs(delta_loss))/(delta_loss.size) < 0.001:
                        break

                    for i, b in enumerate(adj_dict_n2[a]):
                        change = learning_rate * np.matmul(delta_loss[all_ids.index(b),:], za_n3[all_ids.index(b),:])
                        weights_2[a, b] = max(0, min(1, weights_2[a, b] + change))

                    na_za_n2 = [weights_2[a, b]*np.matmul(array_n(eigen_vectors_n2,i), za_n1) for i, b in enumerate(adj_dict_n2[a])]
                    na_za_n2 = sum(na_za_n2)
                    za_n2 = relu(na_za_n2)

                for iterations in range(local_iterations):
                    try:
                        delta_loss = (za_n2 - za_n1) * relu_backwards(za_n2, na_za_n1)
                    except:
                        break

                    if np.sum(np.abs(delta_loss))/(delta_loss.size) < 0.001:
                        break

                    for i, b in enumerate(adj_dict_n2[a]):
                        change = learning_rate * np.matmul(delta_loss[all_ids.index(b),:], za_n2[all_ids.index(b),:])
                        weights_1[a, b] = max(0, min(1, weights_1[a, b] + change))

                    na_za_n1 = [weights_1[a, b]*np.matmul(array_n(laplacian_n1.toarray(),i), conv_strings_n1) for i, b in enumerate(adj_dict_n1[a])]
                    na_za_n1 = sum(na_za_n1)
                    za_n1 = relu(na_za_n1)

        maximum_minutes_validation = int((max(filtered_validation_tweets['created_at']) - min(filtered_validation_tweets['created_at'])).total_seconds()/60)
        for minutes_started in range(0, maximum_minutes_validation, time_length):
            time_slice = filtered_validation_tweets[(filtered_validation_tweets['created_at'] < min(filtered_validation_tweets['created_at']) + datetime.timedelta(minutes=minutes_started+time_length)) & (filtered_validation_tweets['created_at'] >= min(filtered_validation_tweets['created_at']) + datetime.timedelta(minutes=minutes_started))]
            time_slice = time_slice[time_slice['retweeted_user_index'].isin(cluster) & time_slice['retweeter_user_index'].isin(cluster)]

            if len(time_slice) == 0:
                continue

            all_ids = list(set(time_slice['retweeted_user_index'].tolist() + time_slice['retweeter_user_index'].tolist()))

            adj_dict_n1 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 1)
            adj_matrix_n1 = adjacency_dict_to_matrix(all_ids, adj_dict_n1, True)

            adj_dict_n2 = get_nth_neighbour_for_adjacency_dict(create_adjacency_dict(time_slice, 'undirected'), 2)
            adj_matrix_n2 = adjacency_dict_to_matrix(all_ids, adj_dict_n2, True)

            laplacian_n1 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n1)
            eigen_values_n1, eigen_vectors_n1 = np.linalg.eigh(laplacian_n1.toarray())

            laplacian_n2 = undirected_adjacency_dict_to_normalized_laplacian(all_ids, adj_dict_n2)
            eigen_values_n2, eigen_vectors_n2 = np.linalg.eigh(laplacian_n2.toarray())

            for a in all_ids:
                if len(adj_dict_n1[a]) == 0:
                    continue

                za_parts = []
                needed_tweets_n1 = find_all_tweet_connected_to(time_slice, adj_dict_n1[a])
                conv_strings_n1 = array_of_strings_to_float_with_length_sentiment_and_categories(needed_tweets_n1, laplacian_n1.shape[0])

                na_za_n1 = np.array([weights_1[a, b]*np.matmul(array_n(laplacian_n1.toarray(),i), conv_strings_n1) for i, b in enumerate(adj_dict_n1[a])])
                na_za_n1 = sum(na_za_n1)
                za_n1 = relu(na_za_n1)

                na_za_n2 = [weights_2[a, b]*np.matmul(array_n(eigen_vectors_n2,i), za_n1) for i, b in enumerate(adj_dict_n2[a])]
                na_za_n2 = sum(na_za_n2)
                za_n2 = relu(na_za_n2)

                try:
                    delta_loss = (za_n2 - za_n1) * relu_backwards(za_n2, na_za_n1)
                    sum_of_all_losses += np.sum(np.abs(delta_loss))/(delta_loss.size)
                except:
                    continue
        
    print('loss',sum_of_all_losses)
    validation_losses.append(sum_of_all_losses)
    weights_changes.append(weights_1.copy())